In [1]:
import pandas as pd
import numpy as np
import datetime

def str2datetime(date):
    
    date_tmp = date.split('-')
    
    y = int(date_tmp[0])
    m = int(date_tmp[1])
    d = int(date_tmp[2])
    
    return datetime.datetime(y,m,d)

### Load in and combine speed and distance tracking data

In [2]:
speed_dist_7 = pd.read_csv('speed_dist_7days.csv')
speed_dist_7.drop(columns=['Unnamed: 0','GP','Min'],inplace=True)

speed_dist_14 = pd.read_csv('speed_dist_14days.csv')
speed_dist_14.drop(columns=['Unnamed: 0','Team','GP','Min'],inplace=True)

speed_dist_21 = pd.read_csv('speed_dist_21days.csv')
speed_dist_21.drop(columns=['Unnamed: 0','Team','GP','Min'],inplace=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
col_dict_7 = {}
col_dict_14 = {}
col_dict_21 = {}

for c in speed_dist_7.columns[3:]:
    col_dict_7[c] = c+'_Av_7'
    col_dict_14[c] = c+'_Av_14'
    col_dict_21[c] = c+'_Av_21'

In [4]:
speed_dist_7.rename(columns=col_dict_7,inplace=True)
speed_dist_14.rename(columns=col_dict_14,inplace=True)
speed_dist_21.rename(columns=col_dict_21,inplace=True)

In [5]:
same_cols = ['Date','Name']

tmp = speed_dist_7.merge(speed_dist_14,how='inner',left_on=same_cols,right_on=same_cols)
speed_dist_full = tmp.merge(speed_dist_21,how='inner',left_on=same_cols,right_on=same_cols)
speed_dist_full.drop_duplicates(subset=['Name','Date'],inplace=True)

### Load in data with all windows

In [6]:
full_df = pd.read_csv('inj_all_windows.csv')
full_df = full_df.drop(columns='Unnamed: 0')
full_df = full_df.sort_values(by=['Name','Date']).reset_index(drop=True)

In [7]:
for c in full_df.columns:
    print(c)

Name
Height [cm]
Weight [kg]
Position
Pos3
Shoots
Date
Age
Opp
MP
FG
FGA
FG%
3P
3PA
3P%
FT
FTA
FT%
ORB
DRB
TRB
AST
STL
BLK
TOV
PF
PTS
GmSc
+/-
Was_Injured?
Notes
MP_Av7
FG_Av7
FGA_Av7
FG%_Av7
3P_Av7
3PA_Av7
3P%_Av7
FT_Av7
FTA_Av7
FT%_Av7
ORB_Av7
DRB_Av7
TRB_Av7
AST_Av7
STL_Av7
BLK_Av7
TOV_Av7
PF_Av7
PTS_Av7
GmSc_Av7
+/-_Av7
N_games_7
MP_Av14
FG_Av14
FGA_Av14
FG%_Av14
3P_Av14
3PA_Av14
3P%_Av14
FT_Av14
FTA_Av14
FT%_Av14
ORB_Av14
DRB_Av14
TRB_Av14
AST_Av14
STL_Av14
BLK_Av14
TOV_Av14
PF_Av14
PTS_Av14
GmSc_Av14
+/-_Av14
N_games_14
MP_Av21
FG_Av21
FGA_Av21
FG%_Av21
3P_Av21
3PA_Av21
3P%_Av21
FT_Av21
FTA_Av21
FT%_Av21
ORB_Av21
DRB_Av21
TRB_Av21
AST_Av21
STL_Av21
BLK_Av21
TOV_Av21
PF_Av21
PTS_Av21
GmSc_Av21
+/-_Av21
N_games_21
MP_b2b
FG_b2b
FGA_b2b
FG%_b2b
3P_b2b
3PA_b2b
3P%_b2b
FT_b2b
FTA_b2b
FT%_b2b
ORB_b2b
DRB_b2b
TRB_b2b
AST_b2b
STL_b2b
BLK_b2b
TOV_b2b
PF_b2b
PTS_b2b
GmSc_b2b
+/-_b2b
b2b


### Add in column tracking number of previous injuries a player had

In [8]:
N_rows = full_df.shape[0]
N_inj_arr = np.zeros(N_rows)

name = ""
N_injs = 0

for i in range(N_rows):
    
    r = full_df.iloc[i-1] #Look at the previous row, as we want to be careful to not use info from a game to decide
                          #whether or not a player got injured
    
    name_tmp = r['Name']
    
    if name_tmp == name:
        if r['Was_Injured?'] == 1:
            N_injs += 1
    else:
        N_injs = 0
        name = name_tmp
        
    N_inj_arr[i] = N_injs

In [9]:
full_df['N_prev_injs'] = N_inj_arr

In [10]:
full_df.loc[full_df['Was_Injured?']==1].iloc[:10]

,Name,Height [cm],Weight [kg],Position,Pos3,Shoots,Date,Age,Opp,MP,...,AST_b2b,STL_b2b,BLK_b2b,TOV_b2b,PF_b2b,PTS_b2b,GmSc_b2b,+/-_b2b,b2b,N_prev_injs
58,(William) Tony Parker,188,83,Point Guard,Guard,Right,2011-02-27,28.783025,MEM,13.933333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
120,(William) Tony Parker,188,83,Point Guard,Guard,Right,2012-03-07,29.807666,NYK,33.800000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0
125,(William) Tony Parker,188,83,Point Guard,Guard,Right,2012-03-21,29.845996,MIN,10.483333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0
207,(William) Tony Parker,188,83,Point Guard,Guard,Right,2013-02-10,30.736482,BRK,35.316667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0
211,(William) Tony Parker,188,83,Point Guard,Guard,Right,2013-02-22,30.769336,GSW,40.483333,...,7.0,1.0,0.0,0.0,2.0,31.0,30.3,31.0,1.0,4.0
213,(William) Tony Parker,188,83,Point Guard,Guard,Right,2013-03-01,30.788501,SAC,25.300000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5.0
219,(William) Tony Parker,188,83,Point Guard,Guard,Right,2013-04-01,30.873374,MEM,33.300000,...,8.0,0.0,0.0,1.0,1.0,12.0,9.5,-9.0,1.0,6.0
220,(William) Tony Parker,188,83,Point Guard,Guard,Right,2013-04-04,30.881588,OKC,25.450000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,7.0
222,(William) Tony Parker,188,83,Point Guard,Guard,Right,2013-04-14,30.908966,LAL,28.283333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,8.0
259,(William) Tony Parker,188,83,Point Guard,Guard,Right,2013-11-27,31.531143,OKC,34.033333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,9.0


### Add in a column tracking a players annual injury rate per gamae

In [11]:
inj_dict = {}
name = full_df.iloc[0]['Name']
N_injs = 0
N_games = 0
season = "2010-11"
players_failed = []

for i,r in full_df.iterrows():
    
    name_tmp = r['Name']
    date = r['Date']
    date = str2datetime(date)
    yr = date.year if date < datetime.datetime(date.year,8,1) else date.year+1
    
    season_tmp = "{}-{}".format(yr-1,str(yr)[2:])
    
    if name_tmp == name and season_tmp == season:
        N_games +=1
        if r['Was_Injured?'] == 1:
            N_injs += 1   
    else:
        try:
            inj_dict[name,season] = N_injs/N_games
        except ZeroDivisionError:
            players_failed.append(name) #Catch players for which this fails
    
    if name_tmp != name:
        N_games = 1
        N_injs = 1 if r['Was_Injured?']==1 else 0
        name = name_tmp
        season = season_tmp     
    elif season_tmp != season:
        season = season_tmp
        N_games += 1

In [12]:
players_failed

[]

In [13]:
def ann_inj_rate(row):
    
    name = row['Name']
    date = row['Date']
    date = str2datetime(date)
    yr = date.year if date < datetime.datetime(date.year,8,1) else date.year+1
    prev_yr = yr-1
    
    season = "{}-{}".format(prev_yr-1,str(prev_yr)[2:])
    
    try:
        rate = inj_dict[name,season]
        return rate
    except KeyError:
        return 0

In [14]:
full_df['ann_inj_rate'] = full_df.apply(lambda row: ann_inj_rate(row), axis=1)

In [15]:
for c in full_df.columns:
    print(c)

Name
Height [cm]
Weight [kg]
Position
Pos3
Shoots
Date
Age
Opp
MP
FG
FGA
FG%
3P
3PA
3P%
FT
FTA
FT%
ORB
DRB
TRB
AST
STL
BLK
TOV
PF
PTS
GmSc
+/-
Was_Injured?
Notes
MP_Av7
FG_Av7
FGA_Av7
FG%_Av7
3P_Av7
3PA_Av7
3P%_Av7
FT_Av7
FTA_Av7
FT%_Av7
ORB_Av7
DRB_Av7
TRB_Av7
AST_Av7
STL_Av7
BLK_Av7
TOV_Av7
PF_Av7
PTS_Av7
GmSc_Av7
+/-_Av7
N_games_7
MP_Av14
FG_Av14
FGA_Av14
FG%_Av14
3P_Av14
3PA_Av14
3P%_Av14
FT_Av14
FTA_Av14
FT%_Av14
ORB_Av14
DRB_Av14
TRB_Av14
AST_Av14
STL_Av14
BLK_Av14
TOV_Av14
PF_Av14
PTS_Av14
GmSc_Av14
+/-_Av14
N_games_14
MP_Av21
FG_Av21
FGA_Av21
FG%_Av21
3P_Av21
3PA_Av21
3P%_Av21
FT_Av21
FTA_Av21
FT%_Av21
ORB_Av21
DRB_Av21
TRB_Av21
AST_Av21
STL_Av21
BLK_Av21
TOV_Av21
PF_Av21
PTS_Av21
GmSc_Av21
+/-_Av21
N_games_21
MP_b2b
FG_b2b
FGA_b2b
FG%_b2b
3P_b2b
3PA_b2b
3P%_b2b
FT_b2b
FTA_b2b
FT%_b2b
ORB_b2b
DRB_b2b
TRB_b2b
AST_b2b
STL_b2b
BLK_b2b
TOV_b2b
PF_b2b
PTS_b2b
GmSc_b2b
+/-_b2b
b2b
N_prev_injs
ann_inj_rate


### Merge windowed df with speed distance dataframe

In [50]:
cols = ['Date','Name']
injs_speed_dist = full_df.merge(speed_dist_full,how='inner',left_on=cols,right_on=cols)
injs_speed_dist.drop_duplicates(subset=['Name','Date'],inplace=True)

### Drop players who averaged less than 15 minutes per game in our dataset

In [51]:
grouped = injs_speed_dist.groupby(['Name'])
injs_filt = grouped.filter(lambda x: x['MP'].mean()>15)

In [52]:
injs_filt

,Name,Height [cm],Weight [kg],Position,Pos3,Shoots,Date,Age,Opp,MP,...,Avg Speed Def._Av_14,W_Av_21,L_Av_21,Dist. Feet_Av_21,Dist Miles_Av_21,Dist Miles Off._Av_21,Dist Miles Def._Av_21,Avg Speed_Av_21,Avg Speed Off._Av_21,Avg Speed Def._Av_21
0,Aaron Brooks,183,73,Point Guard,Guard,Right,2013-11-01,28.796715,DAL,22.950000,...,4.89,1,0,4499,0.85,0.40,0.45,4.88,4.86,4.89
1,Aaron Brooks,183,73,Point Guard,Guard,Right,2013-11-02,28.799452,UTA,20.566667,...,4.59,2,0,6811.5,1.29,0.65,0.64,4.59,4.60,4.59
2,Aaron Brooks,183,73,Point Guard,Guard,Right,2013-11-04,28.804928,LAC,17.516667,...,4.36,3,0,7077,1.34,0.66,0.68,4.40,4.45,4.36
3,Aaron Brooks,183,73,Point Guard,Guard,Right,2013-11-05,28.807666,POR,1.016667,...,4.29,3,1,6967.5,1.32,0.65,0.67,4.42,4.56,4.29
4,Aaron Brooks,183,73,Point Guard,Guard,Right,2013-11-11,28.824093,TOR,0.100000,...,4.24,4,1,5639.4,1.07,0.52,0.54,4.36,4.48,4.24
5,Aaron Brooks,183,73,Point Guard,Guard,Right,2013-11-13,28.829569,PHI,4.383333,...,4.24,5,1,4709.5,0.89,0.44,0.45,4.43,4.47,4.24
6,Aaron Brooks,183,73,Point Guard,Guard,Right,2013-11-14,28.832307,NYK,0.033333,...,4.33,5,2,4266.3,0.81,0.40,0.41,4.42,4.51,4.33
7,Aaron Brooks,183,73,Point Guard,Guard,Right,2013-11-16,28.837782,DEN,1.000000,...,4.23,6,2,3734.9,0.71,0.35,0.36,4.41,4.51,4.33
8,Aaron Brooks,183,73,Point Guard,Guard,Right,2013-11-19,28.845996,BOS,9.183333,...,4.19,7,2,3353.3,0.64,0.31,0.32,4.36,4.42,4.30
9,Aaron Brooks,183,73,Point Guard,Guard,Right,2013-11-23,28.856947,MIN,24.600000,...,4.28,7,2,3272,0.62,0.31,0.31,4.39,4.48,4.30


### Get rid of columns that correspond to the score in that game, as well as Notes

In [53]:
injs_filt.drop(columns=injs_filt.columns[9:30],inplace=True)
injs_filt.drop(columns='Notes',inplace=True)

In [55]:
for c in injs_filt.columns:
    print(c)

Name
Height [cm]
Weight [kg]
Position
Pos3
Shoots
Date
Age
Opp
Was_Injured?
MP_Av7
FG_Av7
FGA_Av7
FG%_Av7
3P_Av7
3PA_Av7
3P%_Av7
FT_Av7
FTA_Av7
FT%_Av7
ORB_Av7
DRB_Av7
TRB_Av7
AST_Av7
STL_Av7
BLK_Av7
TOV_Av7
PF_Av7
PTS_Av7
GmSc_Av7
+/-_Av7
N_games_7
MP_Av14
FG_Av14
FGA_Av14
FG%_Av14
3P_Av14
3PA_Av14
3P%_Av14
FT_Av14
FTA_Av14
FT%_Av14
ORB_Av14
DRB_Av14
TRB_Av14
AST_Av14
STL_Av14
BLK_Av14
TOV_Av14
PF_Av14
PTS_Av14
GmSc_Av14
+/-_Av14
N_games_14
MP_Av21
FG_Av21
FGA_Av21
FG%_Av21
3P_Av21
3PA_Av21
3P%_Av21
FT_Av21
FTA_Av21
FT%_Av21
ORB_Av21
DRB_Av21
TRB_Av21
AST_Av21
STL_Av21
BLK_Av21
TOV_Av21
PF_Av21
PTS_Av21
GmSc_Av21
+/-_Av21
N_games_21
MP_b2b
FG_b2b
FGA_b2b
FG%_b2b
3P_b2b
3PA_b2b
3P%_b2b
FT_b2b
FTA_b2b
FT%_b2b
ORB_b2b
DRB_b2b
TRB_b2b
AST_b2b
STL_b2b
BLK_b2b
TOV_b2b
PF_b2b
PTS_b2b
GmSc_b2b
+/-_b2b
b2b
N_prev_injs
ann_inj_rate
Team
W_Av_7
L_Av_7
Dist. Feet_Av_7
Dist Miles_Av_7
Dist Miles Off._Av_7
Dist Miles Def._Av_7
Avg Speed_Av_7
Avg Speed Off._Av_7
Avg Speed Def._Av_7
W_Av_14
L_Av_14
D

In [56]:
injs_filt.shape

(133072, 128)

In [57]:
injs_filt.to_csv('inj_data_final.csv')